In [1]:
import pandas as pd

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

from IPython.display import display, Markdown

# Model Input Table

In this document, we aim to demonstrate how to construct a synthetic model input table from other base tables. The input table can be used as part of the CI/CD pipeline to for unit and integration testing.

## Introduction

This example comes from the agricultural sector, using the example of 'fields' (plot of land used to grow crops in) and some features relating to those fields in a specific month (the date dimension).

The features included are:

- nitrogen_applied_kg_per_ha
    - the amount of Nitrogen applied in kgs per ha (ha - hectare, standard unit for areas in agriculture) in that month
- rainfall_mm 
    - observed rainfall in the field in mm that month

## Example:

The example below uses the Python API to construct a base table for the fields, a dimension table for the months and a model input table that uses both tables above (and some features that come from... _somewhere_, just some auxillary info to fill out the example).


In [2]:
import random
import datetime
from tabulate import tabulate
from data_fabricator.v1.core.mock_generator import MockDataGenerator
from data_fabricator.v1.core.mock_generator import (
    BaseTable,
    RandomNumbers,
    UniqueId,
    CrossProduct,
    Date,
    JoinedColumn,
    RowApply,
)


class Fields(BaseTable):
    num_rows = 5
    _metadata_ = {
        "description": "Fields with field area (ha)",
    }
    field_id = UniqueId(
        prefix="field_",
        id_start_range=0,
        id_end_range=5000,
        id_length=10,
        prob_null_kwargs={"seed": 3},
    )
    area_ha = RandomNumbers(
        start_range=5, end_range=50, precision=1, prob_null_kwargs={"seed": 3}
    )


class DateDimension(BaseTable):
    num_rows = 12
    _metadata_ = {"description": "Month indicies, 1...12"}
    date_index = Date(start_dt="2019-01-01", end_dt="2020-01-01", freq="M")


class ModelInputTable(BaseTable):
    _metadata_ = {
        "description": "Model input table with features",
        "dependencies": "DateDimension",
    }

    # Spine
    field_id = CrossProduct(
        check_all_inputs_same_length=False,
        list_of_values=["Fields.field_id", "DateDimension.date_index"],
        position=0,
    )
    date_index = CrossProduct(
        check_all_inputs_same_length=False,
        list_of_values=["Fields.field_id", "DateDimension.date_index"],
        position=1,
    )

    # Joined Column
    field_area_ha = JoinedColumn(
        selected_column="Fields.area_ha",
        this_table_key="ModelInputTable.field_id",
        other_table_key="Fields.field_id",
    )

    # Features
    applied_nitrogen_kg_per_ha = RandomNumbers(
        start_range=150, end_range=350, precision=-1, prob_null_kwargs={"seed": 3}
    )
    rainfall_mm = RowApply(
        list_of_values="ModelInputTable.date_index",
        row_func="lambda x: round(abs(x.month - 7.5) * 10 * random.uniform(0.9 , 1.3),1)",
    )

    # Targets
    # Multiple to show different variations of the target variable you might have.
    growth_mm = RowApply(
        list_of_values=[
            "ModelInputTable.applied_nitrogen_kg_per_ha",
            "ModelInputTable.rainfall_mm",
        ],
        row_func="lambda x,y: 0.06*x + 0.3*y",
    )

Let's break this down.

### Cross Product

The spine of the model input table is formed using the `CrossProduct` of `Fields.field_id` and `Date_Dimension.date_index`, which results in all combinations of the two inputs.


In [3]:
# Spine
field_id = CrossProduct(
    check_all_inputs_same_length=False,
    list_of_values=["Fields.field_id", "DateDimension.date_index"],
    position=0,
)
date_index = CrossProduct(
    check_all_inputs_same_length=False,
    list_of_values=["Fields.field_id", "DateDimension.date_index"],
    position=1,
)

### Joined Column

The field `area_ha` can be joined to the model input table using the `JoinedColumn` class and passing in the column to be joined and the keys to join on.



In [4]:
# Joined Column
field_area_ha = JoinedColumn(
    selected_column="Fields.area_ha",
    this_table_key="ModelInputTable.field_id",
    other_table_key="Fields.field_id",
)

## Result

When executed the following tables are produced:

In [5]:
mdg = MockDataGenerator(tables=[Fields, DateDimension, ModelInputTable])
mdg.generate_all()

for table_name, table in mdg.tables.items():
    print(f"Table: {table_name}")
    print(tabulate(table.dataframe, headers=table.dataframe.columns, tablefmt="psql"))
    print("\n")

Table: Fields
+----+------------+-----------+
|    | field_id   |   area_ha |
|----+------------+-----------|
|  0 | field_1068 |      15.7 |
|  1 | field_1949 |      29.5 |
|  2 | field_3030 |      21.6 |
|  3 | field_4458 |      32.2 |
|  4 | field_4854 |      33.2 |
+----+------------+-----------+


Table: DateDimension
+----+--------------+
|    |   date_index |
|----+--------------|
|  0 |  1.54889e+18 |
|  1 |  1.55131e+18 |
|  2 |  1.55399e+18 |
|  3 |  1.55658e+18 |
|  4 |  1.55926e+18 |
|  5 |  1.56185e+18 |
|  6 |  1.56453e+18 |
|  7 |  1.56721e+18 |
|  8 |  1.5698e+18  |
|  9 |  1.57248e+18 |
| 10 |  1.57507e+18 |
| 11 |  1.57775e+18 |
+----+--------------+


Table: ModelInputTable
+----+------------+---------------------+-----------------+------------------------------+---------------+-------------+
|    | field_id   | date_index          |   field_area_ha |   applied_nitrogen_kg_per_ha |   rainfall_mm |   growth_mm |
|----+------------+---------------------+---------------

## Analysis

Using `r_regression` from `scikit-learn` we can see the correlation between the features and the target.



In [6]:
from sklearn.feature_selection import r_regression

model_input_table = mdg.tables["ModelInputTable"].dataframe

features = ["field_area_ha", "applied_nitrogen_kg_per_ha", "rainfall_mm"]

X = model_input_table[features]
y = model_input_table["growth_mm"]

correlation_matrix = r_regression(X=X, y=y)

print(tabulate([[round(x, 2) for x in correlation_matrix]], headers=features))

  field_area_ha    applied_nitrogen_kg_per_ha    rainfall_mm
---------------  ----------------------------  -------------
           0.11                          0.48           0.88



A quick `DecisionTreeRegression` shows us that even on a relatively small dataset we get a good approximation (funny that...).


In [7]:
# Model Prediction
# This is very likely not the best model to use, but just roll with it,
# and this is for demonstration purposes only.
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

tree_regressor = DecisionTreeRegressor(random_state=0)
tree_regressor.fit(X=X_train, y=y_train)
y_predict = tree_regressor.predict(X=X_test)

In [8]:
fig, ax = plt.subplots()
ax.scatter(y_test, y_predict, s=20, edgecolor="black", c="darkorange", label="data")

# Line
lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, "k-", alpha=0.75, zorder=0)
ax.set_aspect("equal")
ax.set_xlim(lims)
ax.set_ylim(lims)

# Format
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Decision Tree Regression")
plt.legend()

# Show
plt.savefig(
    "data_fabricator/docs/images/8_agriculture_model_input_table.png", format="PNG"
)

plt.clf()
plt.close()

![png](images/8_agriculture_model_input_table.png)


## Conclusion

With this approach we can have some level of control over the correlations that are expected between features and the target. Hence, we can generate a model input table that is suitable for performing a sanity check for an ML pipeline.